In [1]:
# !git clone https://github.com/dsa-playground/tvt2024.git
# %cd /content/tvt2024/
# !git pull
# !pip install upgrade -r requirements.txt -t "tvt2024"
# !pip install pyaml-env

In [2]:
# Imports
import datetime
import warnings
import pandas as pd
from scripts.main import bekijk_data, kies_onderwerp, pas_modellen_toe, pas_gemiddelde_toe, pas_voortschrijdend_gemiddelde_toe, pas_lineaire_regressie_toe, pas_parameters_toe_en_evalueer, voorspel, onderzoek_afwijkingen, bereken_metrieken

## Settings
# settings for pandas
pd.set_option("display.max.columns",None) # alle kolommen tonen
pd.set_option("display.max.rows",500)    # eerste 500 rijen tonen
pd.set_option("display.precision", 2)     # precisie van de kolommen aanpassen
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) # floats output tot 3 decimalen
pd.set_option('display.max_colwidth', None)
datum_vandaag = datetime.datetime.now().strftime("%Y-%m-%d")
# Disable warnings
warnings.filterwarnings('ignore')

# 1. Inleiding

Jullie zijn allen werkzaam bij zorgorganisatie 'Zorgello' in de afdeling Analytics. De bestuurder komt binnen stormen:

Morgen heb ik een vergadering met de Raad van Commisarissen. Vorig jaar hebben ze me flink aan de tand gevoeld. Van een aantal zaken kon ik niet goed aangeven welke richting het op beweegt. Ik heb voor de vergadering morgen wat forecasts nodig. Kunnen jullie deze even snel maken? Het gaat om:
1. Het aantal clienten (ZZP >= 6)
2. Het ziekteverzuim van ons personeel
3. Inzet van flexpool medewerkers

Jullie als afdeling verdelen de onderwerpen. 

Bekijk de data van de verschillende reeksen.

In [3]:
df = bekijk_data()

Wanneer dezelfde variabele op verschillende momenten gemeten worden ontstaat een tijdreeks. In tijdreeksen wordt vaak onderscheid gemaakt tussen:
* Trend
* Seizoenspatroon/-patronen
* Ruis
Afhankelijk van welke tijdreeks je bekijkt zijn er 1 of meerdere van de bovenstaande onderdelen aanwezig. 

**Opgave 1.1.** Een bekende tijdreeks is de temperatuur.

a. Wat weten we van de trend in temperatuur?

<!-- Temperatuur neemt toe door invloed mens: Stijgende trend. -->

b. Welke seizoenspatronen kun je herkennen in de temperatuur?

<!-- - Dagpatroon ('s nachts kouder dan overdag, invloed zonopwarming)
- Jaarpatroon (4 seizoenen, draaing aarde rond de zon) -->

c. Wat zijn factoren die ruis veroorzaken?

<!-- Bewolking, wind- en waterstromingen, zonnevlammen, smog/uitstoot, ... -->

**Opgave 1.2.** Kies een van de tijdreeksen om te voorspellen voor de bestuurder. Door de code in de onderstaande cel te runnen laat je de omgeving weten welke keuze je hebt gemaakt. 


In [4]:
onderwerp = kies_onderwerp()


        Welke reeks wil je voorspellen (a, b, c):
                a. ZZP (verwachting clienten)
                b. Ziekteverzuimpercentage
                c. Flexpool (aantal personen)
        
Gekozen antwoord: Ziekteverzuim


# 2. Modellen

Nu je het onderwerp gekozen hebt, kunnen we een aantal modellen gaan toepassen. 

Een voorspelling is een schatting voor de toekomst. Hoe meer historische, proces en causale informatie, hoe beter een algoritme te bouwen is voor een voorspelling. Om in deze workshop niet te verzanden in verklarende data en complexe algoritmes, hebben we gekozen voor twee ('eenvoudige') modellen:
* Het voortschrijdend gemiddelde
* Lineaire regressie


#### *Het voortschrijdend gemiddelde*
Een ander eenvoudig model om toe te passen is het voortschrijdend gemiddelde. Ook hierbij maak je enkel gebruik van de historische waarden. De formule luidt:

$$
\text{Voortschrijdend gemiddelde} = \frac{\left( a_{n-k+1} + a_{n-k+2} + ... +a_{n}\right)}{k} = \frac{1}{k} \sum_{i=n-k+1}^n a_i
$$

Hierbij is 'a' de historische waarde, 'n' geeft de volgorde van de historische waarde aan en k het aantal voorgaande waarden. Het voortschrijdend gemiddelde is het gemiddelde van de laatste *k* waarden. 

**Opgave 2.1.** Stel het voortschrijdend gemiddelde wordt bepaald op basis van de 4 voorgaande waarden. Hoe zien de eerste 2 getallen (a en b) eruit na deze reeks:

*9, 15, 12, 8, 12, a, b*

a = ...

b = ...


<!-- a = (16 + 12 + 8 + 12) / 4 = 12

b = (12 + 8 + 12 + 12) / 4 = 11 -->


Laten we het voortschrijdend gemiddelde toepassen voor het onderwerp wat jullie gekozen hebben. 


In [5]:
df_voortschrijdend_gemiddelde = pas_voortschrijdend_gemiddelde_toe(df=df, onderwerp=onderwerp)

In de grafiek zijn twee periodes te onderscheiden:
* Het groene vlak is de **trainperiode**. De data uit deze periode is gebruikt om het model te trainen; in dit geval het voortschrijdend gemiddelde te berekenen.
* Het rode vlak is de **testperiode**. De data uit deze periode wordt gebruikt om te toetsen hoe goed het algoritme aansluit bij de gerealiseerde waarden.

**Opgave 2.2.** Wat valt je op aan de uitkomsten van het voortschrijdend gemiddelde model?

<!-- - 1e week in de traindata heeft geen waarde.
- Na een periode van voorspellingen onstaat een constante waarde.  -->


#### *Lineaire regressie*
Er zijn ook algoritmes welke niet enkel kijken naar de historische waarden die je probeert te voorspellen. Deze algoritmes maken gebruik van verklarende variabele(n). Het eenvoudigste model is lineaire regressie. Wat dit model doet een regressielijn bepalen: best passende lijn om de relatie tussen x en y te bepalen. Hierbij is y de variabele waar je naar op zoek bent en x kunnen 1 of meerdere verklarende variabelen zijn. In formulevorm:

$$
y_i = \beta_0 + \beta_1 x_1 + ... + \beta_i x_i + \epsilon_i
$$

Laten we een eenvoudige lineare regressie met 1 variabele (volgordelijkheid datums) toepassen:

In [6]:
df_lineaire_regressie = pas_lineaire_regressie_toe(df=df, onderwerp=onderwerp, yearly_seasonality=False, weekly_seasonality=False)

**Opgave 2.3.** Welke verklarende variabele heeft deze tijdreeks?

<!-- Datumreeks / dag --> deze wordt onder water numeriek gemaakt. -->

# 3. Evalueren

Elk van de modellen geven een voorspelling. Natuurlijk is een visuele check (realisatie naast voorspelling) heel erg belangrijk. Toch is het ook belangrijk om iets meer te kijken naar de afwijking op zichzelf. 

**Opgave 3.1.** Welk model heeft de grootste afwijking?

<!-- Dit is een instinker. De grootte van een afwijking wordt vooral bepaald door wat iemand belangrijk vindt. Temperatuur voorspellen voor kledingkeuze is nog te overzien. Aankopen op de beurs doen kan financieel grote gevolgen hebben. Het gaat dus om hoe nauwkeurig de voorspelling moet zijn op het vervolgproces zo foutloos mogelijk te doen.  -->

#### *De afwijking (error)*
Laten we de reeks van errors voor elk model eens bekijken voor de testperiode.  

In [7]:
onderzoek_afwijkingen(list_of_dfs=[df_voortschrijdend_gemiddelde, df_lineaire_regressie], 
                      onderwerp=onderwerp,
                      start='2023-05-15')

Bij de toegepaste modellen wordt voor de train periode natuurlijk een 'optimum' gezocht binnen de grenzen van een model. Maar afhankelijk van het onderwerp en het model zul je verschillen zien.

**Opgave 3.2.**
Naar welke punten kun je (visueel) kijken bij het beoordelen van de tijdreeks van errors?

<!-- a. Ligt de error rond een bepaalde vaste afwijking? 
b. Neemt de grootte van de afwijking af/toe over tijd? Dit noemen we 'modeldrift'
c. Zijn er extremen te herkennen? Als de error grote afwijkingen kent, dan is er in de oorspronkelijke data heel veel ruis. De vraag is of deze ruis te corrigeren is (in verleden en toekomst). Zo niet, dan moet je je afvragen welke afwijking hinderlijk zijn in het acteren op een voorspelling.  -->

#### *Metrieken*
Om subjectiviteit eruit te halen is het belangrijk stil te staan bij het selecteren van een goede metriek. Een metriek is een getalsmatige uitdrukking van hoe goed/slecht je model presteert. Er zijn ontzettend veel metrieken, elk met een specifiek inzicht en doel. Het toepassen van meerdere metrieken voor de beoordeling van een model wordt ook aangeraden.

Voor deze workshop hebben we 4 metrieken gekozen welke inzicht geven in de prestatie van de modellen:
* **Gemiddelde afwijking**: Voor de test periode worden alle afwijkingen gemiddeld. Let wel: Een gemiddelde van -1 en 1 levert 0 op! Het zegt dus vooral iets over de te verwachte afwijking over langere periode. 
* **Maximale afwijking**: Voor elke dag is er een voorspelling en een realisatie in de testperiode. Deze metriek geeft de maximale afwijking weer. Hiermee zou je de maximale 'schade' kunnen bepalen als een voorspelling gebruikt wordt voor een vervolg(proces).
* **R-squared**: Deze metriek geeft zicht op hoe de relatie is tussen de echte en voorspelde waarden. De uitkomst ligt tussen -1 en 1. Hoe dichter bij de 1 of -1, hoe sterker de relatie en hoe beter het model past. Een score van 0 is de slechtst mogelijke score.
* **MAE**: Mean Absolute Error. Deze metriek berekend het gemiddelde van de verschillen in absolute waarde (alles wordt positief getal). Het geeft daarmee inzicht in hoe groot de gemiddelde afwijking echt is. Hoe dichter bij 0, hoe beter het model presteert. 




In [8]:
bereken_metrieken(list_of_dfs=[df_voortschrijdend_gemiddelde, df_lineaire_regressie], 
                  onderwerp=onderwerp, 
                  start='2023-05-15')

De periode die wordt geanalyseerd is van 2023-05-15 tot 2024-04-15.


,Gemiddelde afwijking,Maximale afwijking,R2-score,Mean Absolute Error
Voortschrijdend gemiddelde,0.450,1.916,-0.429,0.676
Lineaire regressie,-0.031,1.432,0.002,0.582


**Opgave 3.3.** Bij meerdere metrieken kan het voorkomen dat voor sommige metrieken model A beter presenteert en voor andere metrieken model B. Hoe bepaal je dan welk model je moet kiezen? 

<!-- Je kiest metrieken die aansluiten bij je behoefte. Daarnaast is het belangrijk om een volgorde toe te kennen welke metriek je het belangrijkste vindt en hoe groot een afwijking doorslaggevend is. Ook hierbij: Wat is de impact in een vervolgproces!  -->

# 4. Model aanpassen

In de bovenstaande voorbeelden zijn standaard instellingen gekozen. Door het aanpassen van de instellingen, kun je tot een beter model komen. 

Voor beide modellen is er de mogelijkheid om te bepalen welke train- en testperiode je hanteert. Dit doe je met de volgende variabelen:
* vanaf_datum_train_periode: Startdatum van trainperiode in format 'YYYY-MM-DD' (incl. deze dag)
* tot_datum_train_periode: Einddatum van trainperiode in format 'YYYY-MM-DD' (excl. deze dag)
* vanaf_datum_test_periode: Startdatum van testperiode in format 'YYYY-MM-DD' (incl. deze dag)
* tot_datum_test_periode : Startdatum van trainperiode in format 'YYYY-MM-DD' (excl. deze dag)

Daarnaast zijn er specifieke instellingen per model:
* Voortschrijdend gemiddelde:
    - window_size: Het aantal dagen voorafgaand waar het gemiddelde voor berekend wordt. Oftewel, bij 7 wordt het gemiddelde genomen van de voorgaande week. 
    - shift_period: Het aantal dagen dat teruggekeken moet worden. Oftewel, bij 28 (4-wekelijks patroon) gaat het model 4 weken terug en neemt dan de voorgaande window_size om het gemiddelde te bepalen. 

In grafiekvorm:

![Instellingen bij voortschrijdend gemiddelde](https://raw.githubusercontent.com/dsa-playground/tvt2024/main/images/voortschrijdend_gem_instellingen.png)

* Lineaire regressie:
- yearly_seasonality=True
- weekly_seasonality=True
- transformation='spline'
- n_bins=14
- strategy= 'uniform'
- n_knots= 4
- degree= 4


**Opgave 4.1.** Pas de instellingen aan van de modellen en kijk of je tot betere resultaten komt. 

Onderstaande verder gaan: Code opschonen, betere benaming, tekst toevoegen om een en ander toe te lichten

In [9]:
# Kiezen van train- en testperiode
vanaf_datum_train_periode = '2019-01-01'
tot_datum_train_periode = '2023-05-15'
vanaf_datum_test_periode = '2023-05-15'
tot_datum_test_periode = datum_vandaag

# Instellingen model voortschrijdend gemiddelde
window_size = 7
shift_period = 0
predict=False

# Instellingen lineaire regressie
yearly_seasonality=True
weekly_seasonality=True
transformation='spline'
n_bins=14
strategy= 'uniform'
n_knots= 4
degree= 4

In [ ]:
df_voortschrijdend_gemiddelde = pas_voortschrijdend_gemiddelde_toe(
    df=df,
    onderwerp=onderwerp,
    vanaf_datum_train_periode = vanaf_datum_train_periode,
    tot_datum_train_periode = tot_datum_train_periode,
    vanaf_datum_test_periode = vanaf_datum_test_periode,
    tot_datum_test_periode = tot_datum_test_periode,
    window_size= window_size,
    shift_period = shift_period,
    predict = predict,
)

bereken_metrieken(list_of_dfs=[df_voortschrijdend_gemiddelde], 
                  onderwerp=onderwerp, 
                  start='2023-05-15',
                  end=df_voortschrijdend_gemiddelde.index.max())

In [ ]:
df_lineaire_regressie = pas_lineaire_regressie_toe(df=df,
    onderwerp=onderwerp,
    vanaf_datum_train_periode = vanaf_datum_train_periode,
    tot_datum_train_periode = tot_datum_train_periode,
    vanaf_datum_test_periode = vanaf_datum_test_periode,
    tot_datum_test_periode = tot_datum_test_periode,
    yearly_seasonality = yearly_seasonality,
    weekly_seasonality = weekly_seasonality,
    transformation=transformation, n_bins=n_bins, strategy=strategy, n_knots=n_knots, degree=degree
)

bereken_metrieken(list_of_dfs=[df_lineaire_regressie], 
                  onderwerp=onderwerp, 
                  start='2023-05-15',
                  end=df_lineaire_regressie.index.max())

In [ ]:
# # Stap 5: Optimaliseren van je algoritme ---> Vervangen door functie per model. 

# # Generiek voor alle modellen
# vanaf_datum_train_periode = '2019-01-01'
# tot_datum_train_periode = '2023-05-15'
# vanaf_datum_test_periode = '2023-05-15'
# tot_datum_test_periode = '2024-05-15'

# # Gemiddelde
# # - Geen specifieke parameters

# # Voortschrijdend_gemiddelde
# window_size = 7
# shift_period = 365
# predict=False

# # Lineaire regressie
# yearly_seasonality=True
# weekly_seasonality=True
# transformation='spline'
# n_bins=14
# strategy= 'uniform'
# n_knots= 4
# degree= 4

# df_evalueer = pas_parameters_toe_en_evalueer(
#     df=df,
#     onderwerp=onderwerp,
#     vanaf_datum_train_periode = vanaf_datum_train_periode,
#     tot_datum_train_periode = tot_datum_train_periode,
#     vanaf_datum_test_periode = vanaf_datum_test_periode,
#     tot_datum_test_periode = tot_datum_test_periode,
#     window_size= window_size,
#     shift_period = shift_period,
#     predict = predict,
#     yearly_seasonality = yearly_seasonality,
#     weekly_seasonality = weekly_seasonality,
#     transformation=transformation, n_bins=n_bins, strategy=strategy, n_knots=n_knots, degree=degree
# )

# bereken_metrieken(df=df_evalueer, 
#                   onderwerp=onderwerp, 
#                   start=vanaf_datum_test_periode,
#                   end=df_evalueer.index.max())

Voorspelfunctie nog aanpassen zodat een model gekozen kan worden

In [ ]:
# Stap 6: Voorspel!
# voorspel()

# Generiek voor alle modellen
vanaf_datum_train_periode = '2019-01-01'
tot_datum_train_periode = '2023-05-15'
vanaf_datum_test_periode = '2023-05-15'
tot_datum_test_periode = '2026-01-01'

# Gemiddelde
# - Geen specifieke parameters

# Voortschrijdend_gemiddelde
window_size = 7
shift_period = 365
predict=False

# Lineaire regressie
yearly_seasonality=True
weekly_seasonality=True
transformation='spline'
n_bins=14
strategy= 'uniform'
n_knots= 4
degree= 4

df_toekomst = voorspel(
    df=df,
    onderwerp=onderwerp,
    vanaf_datum_train_periode = vanaf_datum_train_periode,
    tot_datum_train_periode = tot_datum_train_periode,
    vanaf_datum_test_periode = vanaf_datum_test_periode,
    tot_datum_test_periode = tot_datum_test_periode,
    window_size= window_size,
    shift_period = shift_period,
    predict = predict,
    yearly_seasonality = yearly_seasonality,
    weekly_seasonality = weekly_seasonality,
    transformation=transformation, n_bins=n_bins, strategy=strategy, n_knots=n_knots, degree=degree, shapes=False
)

# Belangrijke les om mee te nemen is het proces wat je opzet! 
# Dit proces is herhaalbaar en kan je steeds verbeteren. 
# Gebruik elke keer de resultaten van de vorige stap om de volgende stap te verbeteren.
# Corrigeer outliers indien bekend.

In [ ]:
# TODO:

# DONE: Fix shapes in plot functie (nu hardcoded 1 jaar van max)
# DONE: Verplaatsen functies naar preprocess, evaluate en model
# DONE: Requirements uitwerken
# DONE: Push naar github
# DONE: Test op colab
# Tekst toevoegen aan notebook
# Presentaie aanvullen met toelichting

# DONE: R2 toevoegen aan metrieken?
# Na workshop: Optimum uitrekenen voor elk dataset en model
# Code opschonen
# 